In [2]:
import heapq  # Importing heapq to use a priority queue for the open list

# Define the grid with 0s as free spaces and 1s as obstacles
grid = [
    [0, 0, 0, 0, 1],
    [1, 1, 0, 1, 1],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 0, 0],
    [0, 0, 1, 1, 0]
]

# Heuristic function using Manhattan Distance
def manhattan_distance(x1, y1, x2, y2):
    # Calculate the Manhattan distance between two points (x1, y1) and (x2, y2)
    return abs(x1 - x2) + abs(y1 - y2)

# A* Algorithm for pathfinding
def a_star_algorithm(grid, start, end):
    # Get the number of rows and columns in the grid
    rows, cols = len(grid), len(grid[0])

    # Open list (priority queue) initialized with the starting node (f_score, (x, y))
    open_list = []
    heapq.heappush(open_list, (0, start))  # f_score = 0 at the start node

    # Dictionaries to track the shortest known path and the cost of getting to a node
    came_from = {}  # Tracks the parent of each node for path reconstruction
    g_score = {start: 0}  # Tracks the cost from start to the current node
    f_score = {start: manhattan_distance(*start, *end)}  # f_score = g_score + heuristic

    # Movement directions (right, down, left, up)
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    # Main loop that continues as long as there are nodes to explore
    while open_list:
        # Pop the node with the lowest f_score (best candidate for exploration)
        current = heapq.heappop(open_list)[1]

        # If we reached the destination, reconstruct the path and return it
        if current == end:
            path = []  # List to store the path
            while current in came_from:
                path.append(current)
                current = came_from[current]  # Move to the parent node
            path.append(start)
            path.reverse()  # Reverse the path to get it from start to end
            return path

        # Explore the neighbors of the current node
        for direction in directions:
            # Calculate the neighbor's coordinates
            neighbor = (current[0] + direction[0], current[1] + direction[1])

            # Ensure the neighbor is within the grid bounds and not an obstacle
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] == 0:
                tentative_g_score = g_score[current] + 1  # Tentative g_score if we move to the neighbor

                # If this path to neighbor is better than any previously found, update
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current  # Record the best path to the neighbor
                    g_score[neighbor] = tentative_g_score  # Update g_score for the neighbor
                    f_score[neighbor] = tentative_g_score + manhattan_distance(*neighbor, *end)  # Update f_score

                    # Add the neighbor to the open list to explore later
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))

    return None  # If no path is found, return None

# Function to mark the path on the grid with 'P'
def mark_path_on_grid(grid, path):
    # Iterate over the path and mark the cells with 'P'
    for (x, y) in path:
        grid[x][y] = 'P'
    grid[0][0] = 'P'  # Mark the start point
    grid[4][4] = 'P'  # Mark the end point
    return grid

# Define the start and end points on the grid
start = (0, 0)
end = (4, 4)

# Run the A* algorithm to find the shortest path
path = a_star_algorithm(grid, start, end)

# If a path is found, mark it on the grid and print the updated grid
if path:
    grid_with_path = mark_path_on_grid([row[:] for row in grid], path)  # Copy the grid to avoid modifying the original

    # Print the grid with the path marked
    for row in grid_with_path:
        print(' '.join(str(cell) for cell in row))
else:
    print("No path found.")


P P P 0 1
1 1 P 1 1
0 0 P 1 0
1 1 P P P
0 0 1 1 P


In [3]:
import heapq

# Heuristic function that calculates the absolute difference
# between the current amount of water in either jug and the target (2 liters)
def heuristic(jug1, jug2, target=2):
    return abs(jug1 - target) + abs(jug2 - target)

# A* algorithm to solve the water-jug problem
def a_star_water_jug(jug1_capacity, jug2_capacity, target):
    # Priority queue for the open list
    open_list = []
    # Starting state (0 liters in both jugs)
    start_state = (0, 0)
    # The initial cost (g_score) is 0
    heapq.heappush(open_list, (0, start_state))

    # Keep track of the path taken to each state
    came_from = {}
    # Dictionary to keep the g_score (cost to reach each state)
    g_score = {start_state: 0}

    # Set to keep track of visited states
    visited = set()

    # Possible actions (states) from any current state
    def get_neighbors(state):
        jug1, jug2 = state
        neighbors = []

        # Fill jug 1
        neighbors.append((jug1_capacity, jug2))
        # Fill jug 2
        neighbors.append((jug1, jug2_capacity))

        # Empty jug 1
        neighbors.append((0, jug2))
        # Empty jug 2
        neighbors.append((jug1, 0))

        # Pour jug 1 into jug 2
        transfer = min(jug1, jug2_capacity - jug2)
        neighbors.append((jug1 - transfer, jug2 + transfer))

        # Pour jug 2 into jug 1
        transfer = min(jug2, jug1_capacity - jug1)
        neighbors.append((jug1 + transfer, jug2 - transfer))

        return neighbors

    # Main A* loop
    while open_list:
        # Get the state with the lowest f_score (heuristic + g_score)
        _, current_state = heapq.heappop(open_list)

        # If either jug has exactly 2 liters, return the solution path
        if current_state[0] == target or current_state[1] == target:
            # Reconstruct the path by tracing back through the came_from dictionary
            path = []
            while current_state in came_from:
                path.append(current_state)
                current_state = came_from[current_state]
            path.append(start_state)
            path.reverse()
            return path

        # Mark the current state as visited
        visited.add(current_state)

        # Get the neighboring states (possible next moves)
        for neighbor in get_neighbors(current_state):
            if neighbor in visited:
                continue

            # Calculate the tentative g_score for the neighbor
            tentative_g_score = g_score[current_state] + 1

            # If this path is better than any previously found, update
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current_state
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic(neighbor[0], neighbor[1], target)
                heapq.heappush(open_list, (f_score, neighbor))

    return None  # No solution found

# Define jug capacities and the target amount of water
jug1_capacity = 4
jug2_capacity = 3
target = 2

# Solve the water-jug problem using A* algorithm
solution_path = a_star_water_jug(jug1_capacity, jug2_capacity, target)

# Output the solution path
if solution_path:
    print("Solution Path (Jug 1, Jug 2):")
    for state in solution_path:
        print(state)
else:
    print("No solution found.")


Solution Path (Jug 1, Jug 2):
(0, 0)
(0, 3)
(3, 0)
(3, 3)
(4, 2)


In [6]:
import random

# Function to calculate the number of conflicts (pairs of queens attacking each other)
def calculate_conflicts(queens):
    conflicts = 0
    n = len(queens)

    for i in range(n):
        for j in range(i + 1, n):
            # Check if queens i and j are in the same column or on the same diagonal
            if queens[i] == queens[j] or abs(queens[i] - queens[j]) == j - i:
                conflicts += 1
    return conflicts

# Function to generate a random initial state (solution)
def generate_initial_state(n):
    return [random.randint(0, n - 1) for _ in range(n)]

# Hill-Climbing algorithm to solve the 8-queen problem
def hill_climbing(n):
    # Generate a random initial state
    current_state = generate_initial_state(n)
    current_conflicts = calculate_conflicts(current_state)

    while current_conflicts > 0:
        # Generate all possible neighbors (states)
        neighbors = []

        for col in range(n):
            for row in range(n):
                if row != current_state[col]:  # Move queen in column `col` to `row`
                    neighbor = current_state[:]
                    neighbor[col] = row
                    neighbors.append(neighbor)

        # Find the best neighbor (the one with the minimum conflicts)
        best_neighbor = None
        best_conflicts = current_conflicts

        for neighbor in neighbors:
            conflicts = calculate_conflicts(neighbor)
            # If this neighbor has fewer conflicts, consider it as the best neighbor
            if conflicts < best_conflicts:
                best_conflicts = conflicts
                best_neighbor = neighbor

        # If the best neighbor is not improving, we have reached a peak
        if best_conflicts >= current_conflicts:
            break

        # Move to the best neighbor
        current_state = best_neighbor
        current_conflicts = best_conflicts

    # Return the solution (or None if no solution found)
    if current_conflicts == 0:
        return current_state
    else:
        return None

# Solve the 8-queen problem using Hill-Climbing
solution = hill_climbing(8)

# Output the solution
if solution:
    print("Solution found:")
    for row in solution:
        line = ['.'] * 8  # Create an empty row
        line[row] = 'Q'  # Place a queen
        print(' '.join(line))  # Print the row
else:
    print("No solution found.")


No solution found.


In [ ]:
import numpy as np

# Constants for players
PLAYER_X = 'X'
PLAYER_O = 'O'
EMPTY = ' '

# Function to print the Tic-Tac-Toe board
def print_board(board):
    for row in board:
        print('|'.join(row))
        print('-' * 5)

# Function to check for a win
def check_winner(board):
    # Check rows, columns, and diagonals for a win
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] != EMPTY:
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] != EMPTY:
            return board[0][i]

    if board[0][0] == board[1][1] == board[2][2] != EMPTY:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] != EMPTY:
        return board[0][2]

    return None  # No winner yet

# Function to check if the board is full (draw condition)
def is_draw(board):
    return all(cell != EMPTY for row in board for cell in row)

# Mini-Max algorithm to choose the best move
def mini_max(board, depth, is_maximizing):
    winner = check_winner(board)
    if winner == PLAYER_X:
        return 1  # X wins
    elif winner == PLAYER_O:
        return -1  # O wins
    elif is_draw(board):
        return 0  # Draw

    if is_maximizing:
        best_score = -np.inf  # Initialize to negative infinity
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X  # Make the move
                    score = mini_max(board, depth + 1, False)  # Recursively call mini-max
                    board[i][j] = EMPTY  # Undo the move
                    best_score = max(score, best_score)  # Update the best score
        return best_score
    else:
        best_score = np.inf  # Initialize to positive infinity
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O  # Make the move
                    score = mini_max(board, depth + 1, True)  # Recursively call mini-max
                    board[i][j] = EMPTY  # Undo the move
                    best_score = min(score, best_score)  # Update the best score
        return best_score

# Function to find the best move for the computer
def find_best_move(board):
    best_score = -np.inf
    move = (-1, -1)

    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                board[i][j] = PLAYER_X  # Make the move
                score = mini_max(board, 0, False)  # Get the score
                board[i][j] = EMPTY  # Undo the move

                if score > best_score:  # Check if this move is better
                    best_score = score
                    move = (i, j)  # Update the best move

    return move

# Main function to play the game
def play_game():
    board = [[EMPTY for _ in range(3)] for _ in range(3)]  # Initialize the board
    print("Welcome to Tic-Tac-Toe!")
    print_board(board)

    while True:
        # Player's turn (O)
        while True:
            row = int(input("Enter the row (0, 1, 2) for your move: "))
            col = int(input("Enter the column (0, 1, 2) for your move: "))
            if board[row][col] == EMPTY:
                board[row][col] = PLAYER_O  # Make the move
                break
            else:
                print("Invalid move! Try again.")

        print_board(board)
        if check_winner(board) == PLAYER_O:
            print("You win!")
            break
        if is_draw(board):
            print("It's a draw!")
            break

        # Computer's turn (X)
        print("Computer's turn (X)...")
        move = find_best_move(board)
        if move != (-1, -1):
            board[move[0]][move[1]] = PLAYER_X  # Make the move
            print_board(board)
            if check_winner(board) == PLAYER_X:
                print("Computer wins!")
                break
            if is_draw(board):
                print("It's a draw!")
                break

# Run the Tic-Tac-Toe game
if __name__ == "__main__":
    play_game()


Welcome to Tic-Tac-Toe!
 | | 
-----
 | | 
-----
 | | 
-----
Enter the row (0, 1, 2) for your move: 2
Enter the column (0, 1, 2) for your move: 1
 | | 
-----
 | | 
-----
 |O| 
-----
Computer's turn (X)...
 |X| 
-----
 | | 
-----
 |O| 
-----
Enter the row (0, 1, 2) for your move: 1
Enter the column (0, 1, 2) for your move: 0
 |X| 
-----
O| | 
-----
 |O| 
-----
Computer's turn (X)...
 |X| 
-----
O| | 
-----
X|O| 
-----
Enter the row (0, 1, 2) for your move: 2
Enter the column (0, 1, 2) for your move: 2
 |X| 
-----
O| | 
-----
X|O|O
-----
Computer's turn (X)...
 |X|X
-----
O| | 
-----
X|O|O
-----
Enter the row (0, 1, 2) for your move: 2
Enter the column (0, 1, 2) for your move: 1
Invalid move! Try again.
